In [ ]:
import albumentations as alb
import copy
import time
import os
import cv2 
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=640, height=360),
                         alb.HorizontalFlip(p=0.5),
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='yolo', 
                                                  label_fields=['class_labels'],
                                                  min_area=1024, #remove this if your objects are small
                                                  min_visibility=0.3)) # remove this contraint if your objects are small

In [ ]:
# Your "Main_folder" should contain two folders "orignal" and "augmented"
# "orignal" and "augmented" folders should contain two sub-folders, "images" and "labels" folder 

# "orignal" folder should contain data to be augmented.

for image in os.listdir(os.path.join("Main_folder","orignal","images")): 
    orignal_image = cv2.imread(os.path.join("Main_folder","orignal","images",image))
    bbox_file=os.path.join("Main_folder","orignal","labels", image.split(".")[0]+".txt")
    
    orignal_image = cv2.cvtColor(orignal_image, cv2.COLOR_BGR2RGB)
    
    org_bbox=[]
    class_labels =[]
    
    if os.path.exists(bbox_file):
        bbox_file =open(bbox_file,"r")
        for line in bbox_file:
            org_bbox.append(list(map(float,line.split())))
        bbox_file.close()
    else:
        continue
    
    
    for col in org_bbox:
        class_labels.append(col[0])
        del col[0]
    
    try:
        for x in range(60):
            print(class_labels)
            augmented = augmentor(image=orignal_image, bboxes=org_bbox, class_labels=class_labels)
            #print(augmented['class_labels'])
            cv2.imwrite(os.path.join("Main_folder","augmented","images",f'{image.split(".")[0]}_{x}.jpg'), augmented['image'])
            txt_file = open(os.path.join("Main_folder","augmented","labels",f'{image.split(".")[0]}_{x}.txt'),"w")
            
            for items in augmented['bboxes']:
                label = 0
                txt_file.write(str(int(augmented['class_labels'][label])))
                txt_file.write(" ")
                for n in items:
                    txt_file.write(str(n))
                    txt_file.write(" ")
                if label < len(augmented['bboxes']) :
                    txt_file.write("\n")
                label += 1
            
            txt_file.close()
                
    except Exception as e:
        print(e)
            